# **Project: Football Data Analysis**

## Table of Contents

# **Introduction**

---

> This is an extensive dataset of football/soccer teams, players, leagues and matches played for the duration of 2008 to 2016. It is a dataset suited for machine learning but I am just performing exploratory data analysis with it. As a result, I will not be making use of a lot of the data from this dataset.
> 
> After studying the tables, I have come up with some questions that I plan to explore:
> 
> 1. 

# **Loading The Data**

---


### Importing all modules and packages that will be used


In [336]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sqlite3
from sqlalchemy import create_engine

import pprint

%matplotlib inline

### Connecting to the SQLite database


In [337]:
# This code connects to the database through sqlite3
# This enables the database to be queried with traditional SQL code
load = sqlite3.connect('database.sqlite')
cursor = load.cursor()

# This code connects to the database through sqlalchemy
# This enables the database tables to be stored in a pandas dataframe and analyzed with python code and packages
engine = create_engine('sqlite:///database.sqlite')


### Listing all tables in the database


In [338]:
# This code checks the database to list all tables inside
cursor.execute('SELECT name from sqlite_master where type= "table"')
pprint.pprint(cursor.fetchall())


[('sqlite_sequence',),
 ('Player_Attributes',),
 ('Player',),
 ('Match',),
 ('League',),
 ('Country',),
 ('Team',),
 ('Team_Attributes',)]


> Even though the above list contains 8 tables, there are actually 7 tables in the database. The first table `sqlite_sequence` is automatically generated by [SQLite](https://www.sqlite.org/autoinc.html#:~:text=The%20sqlite_sequence%20table%20is%20created,%2C%20INSERT%2C%20and%20DELETE%20statements.) to keep track of the other tables.


### Creating dataframes for each of the 7 tables


In [339]:
df_league = pd.read_sql_table("League", engine)
df_player = pd.read_sql_table("Player", engine)
df_match = pd.read_sql_table("Match", engine)
df_country = pd.read_sql_table("Country", engine)
df_team = pd.read_sql_table("Team", engine)
df_team_attr = pd.read_sql_table("Team_Attributes", engine)
df_player_attr = pd.read_sql_table("Player_Attributes", engine)

# cursor.close()


_From this point on, I try to run my analysis in both Python and SQL code._


# **Wrangling The Data**

---


## **A. `Match` Table**


In [340]:
df_match.head()


,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67


### Removing Columns

> The `Match` table is very large and contains many columns which I would not need for my current analysis (especially the betting odds columns), some columns are missing all their values and some columns are not explained in the documentation so I don't know what they represent. I drop all irrelevant columns.
>
> I will be removing:
>
> 1. all columns that have **player** in their name, because I do not know what those numbers represent and it is not explained on the documentation
> 2. all the betting odds columns at the right side of the table, because I am not using them for my analysis
> 3. all the match statistics colummns (except the goals scored), because the values are not available in the dataset, they are all missing


In [341]:
# Listing all column names in the `match` table so I can know the index of the columns to be deleted
list(enumerate(df_match.columns, start=0))

[(0, 'id'),
 (1, 'country_id'),
 (2, 'league_id'),
 (3, 'season'),
 (4, 'stage'),
 (5, 'date'),
 (6, 'match_api_id'),
 (7, 'home_team_api_id'),
 (8, 'away_team_api_id'),
 (9, 'home_team_goal'),
 (10, 'away_team_goal'),
 (11, 'home_player_X1'),
 (12, 'home_player_X2'),
 (13, 'home_player_X3'),
 (14, 'home_player_X4'),
 (15, 'home_player_X5'),
 (16, 'home_player_X6'),
 (17, 'home_player_X7'),
 (18, 'home_player_X8'),
 (19, 'home_player_X9'),
 (20, 'home_player_X10'),
 (21, 'home_player_X11'),
 (22, 'away_player_X1'),
 (23, 'away_player_X2'),
 (24, 'away_player_X3'),
 (25, 'away_player_X4'),
 (26, 'away_player_X5'),
 (27, 'away_player_X6'),
 (28, 'away_player_X7'),
 (29, 'away_player_X8'),
 (30, 'away_player_X9'),
 (31, 'away_player_X10'),
 (32, 'away_player_X11'),
 (33, 'home_player_Y1'),
 (34, 'home_player_Y2'),
 (35, 'home_player_Y3'),
 (36, 'home_player_Y4'),
 (37, 'home_player_Y5'),
 (38, 'home_player_Y6'),
 (39, 'home_player_Y7'),
 (40, 'home_player_Y8'),
 (41, 'home_player_Y9'),
 (

In [342]:
# Creating a function to store all columns that will be removed from the Match table
def match_columns_to_delete():
    # Storing the table columns in a variable
    match_columns = df_match.columns

    # Creating a list of all columns to be deleted
    # match_columns_to_delete = [head for head in match_columns if head.find('player')>0 or head.find('365')>0 or len(head)==3]
    match_columns_to_delete = match_columns[11:]

    # Confirming changes
    return match_columns_to_delete


In [343]:
# Dropping unwanted columns
df_match.drop(columns=match_columns_to_delete(), inplace=True)


In [344]:
# Confirming changes
df_match.head()


,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,1
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,0
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,3
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,0
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,3


### Reorganizing The Table
> Here I will be making the following changes:
> 1. Join the `team_api_id` from 2 columns to one column. 
> 2. Join `team_goals` from 2 columns to one column
>
>I will achieve this by making 2 seperate tables, one for home and one for away. Then joining the tables at the end.

In [345]:
# Creating a function to store the dataframe of only home matches
def home_matches():
    # Creating a table for home team by dropping away team columns
    df_match_h = df_match.drop(columns=['away_team_api_id', 'away_team_goal'])

    # Adding a column to specifty these are home matches
    df_match_h['side'] = 'home'

    # Renaming the columns since I've seperated the home and away matches
    df_match_h.rename(columns={'home_team_api_id':'team_api_id', 'home_team_goal':'team_goal'}, inplace = True)

    return df_match_h

# Confirming changes
home_matches().head()

,id,country_id,league_id,season,stage,date,match_api_id,team_api_id,team_goal,side
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,1,home
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,0,home
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,0,home
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,5,home
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,1,home


In [346]:
# Creating a function to store the dataframe of only away matches
def away_matches():
    # Creating a table for away team by dropping home team columns
    df_match_a = df_match.drop(columns=['home_team_api_id', 'home_team_goal'])

    # Adding a column to specifty these are away matches
    df_match_a['side'] = 'away'

    # Renaming the columns since I've seperated the home and away matches
    df_match_a.rename(columns={'away_team_api_id':'team_api_id', 'away_team_goal':'team_goal'}, inplace = True)

    return df_match_a

# Confirming changes
away_matches()

,id,country_id,league_id,season,stage,date,match_api_id,team_api_id,team_goal,side
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9993,1,away
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,9994,0,away
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,8635,3,away
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9998,0,away
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,9985,3,away
...,...,...,...,...,...,...,...,...,...,...
25974,25975,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992091,10191,0,away
25975,25976,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992092,10199,2,away
25976,25977,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992093,10179,0,away
25977,25978,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992094,10243,0,away


### Joining the tables

In [347]:
# Joining the 2 tables and storing the dataframe in a new table so that the original table is still there for reference purposes
df_match_ed = pd.concat([home_matches(), away_matches()])

# Confirming changes
df_match_ed.head()

,id,country_id,league_id,season,stage,date,match_api_id,team_api_id,team_goal,side
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,1,home
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,0,home
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,0,home
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,5,home
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,1,home


### Adding Columns
> Here I will be making changes to the table by doing the following:
> 1. Adding the country names by referencing the country table
> 2. Adding the league names by referencing the league table
> 3. Adding the team names by referencnig the team table
> 
> Lets take start with the Country table

In [348]:
df_country.head(2)

,id,name
0,1,Belgium
1,1729,England


> The `id` column is the foreign key I will be using to get the country names that will be inserted to the match table so I will need to change the `id` column name to match with the `country_id` column in the edited match table so that they have the same column name

In [349]:
# Renaming the columns
df_country.rename(columns={'id':'country_id', 'name':'country'}, inplace=True)

# Confirming changes
df_country

,country_id,country
0,1,Belgium
1,1729,England
2,4769,France
3,7809,Germany
4,10257,Italy
5,13274,Netherlands
6,15722,Poland
7,17642,Portugal
8,19694,Scotland
9,21518,Spain


In [350]:
# Merging the tables using a left-join
df_match_ed = pd.merge(df_match_ed, df_country, on='country_id', how='left')

# Confirming changes
df_match_ed.head()

,id,country_id,league_id,season,stage,date,match_api_id,team_api_id,team_goal,side,country
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,1,home,Belgium
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,0,home,Belgium
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,0,home,Belgium
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,5,home,Belgium
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,1,home,Belgium


> Next I check the league table

In [351]:
df_league.head(2)

,id,country_id,name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League


> Again the `id` column is the foreign key I will be using to get the league names that will be inserted to the match table so I will need to change the `id` column name to match with the `league_id` column in the edited match table so that they have the same column name

In [352]:
# Renaming the columns
df_league.rename(columns={'id':'league_id', 'name':'league'}, inplace=True)

# Confirming changes
df_league.head(2)

,league_id,country_id,league
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League


In [353]:
# Merging the tables using a left-join
df_match_ed = pd.merge(df_match_ed, df_league, on='league_id', how='left')

# Confirming changes
df_match_ed.head()

,id,country_id_x,league_id,season,stage,date,match_api_id,team_api_id,team_goal,side,country,country_id_y,league
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,1,home,Belgium,1,Belgium Jupiler League
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,0,home,Belgium,1,Belgium Jupiler League
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,0,home,Belgium,1,Belgium Jupiler League
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,5,home,Belgium,1,Belgium Jupiler League
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,1,home,Belgium,1,Belgium Jupiler League


> Some extra columns were created because the match table and the league table both have a column for `country_id` but that will not be a problem as I will be dropping those columns soon.
>
> Now lets check the team table.

In [354]:
df_team

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB
...,...,...,...,...,...
294,49479,10190,898.0,FC St. Gallen,GAL
295,49837,10191,1715.0,FC Thun,THU
296,50201,9777,324.0,Servette FC,SER
297,50204,7730,1862.0,FC Lausanne-Sports,LAU


> Here the foreign key `team_api_id` is the same as the Match table so I'll only be changing the team name column. I will be working with the team's full name (not the short name).

In [355]:
# Renaming the columns
df_team.rename(columns={'team_long_name':'team'}, inplace=True)

# Confirming changes
df_team.head(2)

,id,team_api_id,team_fifa_api_id,team,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC


In [356]:
# Merging the tables using a left-join
df_match_ed = pd.merge(df_match_ed, df_team, on='team_api_id', how='left')

# Confirming changes
df_match_ed.head()

,id_x,country_id_x,league_id,season,stage,date,match_api_id,team_api_id,team_goal,side,country,country_id_y,league,id_y,team_fifa_api_id,team,team_short_name
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,1,home,Belgium,1,Belgium Jupiler League,1,673.0,KRC Genk,GEN
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,0,home,Belgium,1,Belgium Jupiler League,3,15005.0,SV Zulte-Waregem,ZUL
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,0,home,Belgium,1,Belgium Jupiler League,5,1750.0,KSV Cercle Brugge,CEB
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,5,home,Belgium,1,Belgium Jupiler League,7,674.0,KAA Gent,GEN
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,1,home,Belgium,1,Belgium Jupiler League,9,NaN,FCV Dender EH,DEN


> The tables have been merged and it joined all the columns from the other tables. Now I will be dropping all unwanted columns.

In [357]:
# Dropping the columns
df_match_ed.drop(columns=['id_x', 'country_id_x', 'league_id', 'match_api_id', 'team_api_id', 'country_id_y', 'id_y', 'team_fifa_api_id', 'team_short_name'], inplace=True)

# Confirming changes
df_match_ed

,season,stage,date,team_goal,side,country,league,team
0,2008/2009,1,2008-08-17 00:00:00,1,home,Belgium,Belgium Jupiler League,KRC Genk
1,2008/2009,1,2008-08-16 00:00:00,0,home,Belgium,Belgium Jupiler League,SV Zulte-Waregem
2,2008/2009,1,2008-08-16 00:00:00,0,home,Belgium,Belgium Jupiler League,KSV Cercle Brugge
3,2008/2009,1,2008-08-17 00:00:00,5,home,Belgium,Belgium Jupiler League,KAA Gent
4,2008/2009,1,2008-08-16 00:00:00,1,home,Belgium,Belgium Jupiler League,FCV Dender EH
...,...,...,...,...,...,...,...,...
51953,2015/2016,9,2015-09-22 00:00:00,0,away,Switzerland,Switzerland Super League,FC Thun
51954,2015/2016,9,2015-09-23 00:00:00,2,away,Switzerland,Switzerland Super League,FC Luzern
51955,2015/2016,9,2015-09-23 00:00:00,0,away,Switzerland,Switzerland Super League,FC Sion
51956,2015/2016,9,2015-09-22 00:00:00,0,away,Switzerland,Switzerland Super League,FC Zürich


### Check for duplicate rows

In [358]:
# Checking for duplicate rows
df_match_ed.duplicated().sum()

3

In [359]:
# Having a look at all the duplicates
df_match_ed[df_match_ed.duplicated(keep=False)]

,season,stage,date,team_goal,side,country,league,team
16347,2010/2011,26,2011-05-10 00:00:00,1,home,Poland,Poland Ekstraklasa,Polonia Bytom
16350,2010/2011,26,2011-05-10 00:00:00,1,home,Poland,Poland Ekstraklasa,Polonia Bytom
16409,2010/2011,6,2010-09-18 00:00:00,1,home,Poland,Poland Ekstraklasa,Polonia Bytom
16415,2010/2011,6,2010-09-18 00:00:00,1,home,Poland,Poland Ekstraklasa,Polonia Bytom
42317,2010/2011,25,2011-05-07 00:00:00,0,away,Poland,Poland Ekstraklasa,Polonia Bytom
42323,2010/2011,25,2011-05-07 00:00:00,0,away,Poland,Poland Ekstraklasa,Polonia Bytom


In [360]:
# Dropping duplicates since there are only 3
df_match_ed = df_match_ed.drop_duplicates()

# Confirming changes
df_match_ed.duplicated().sum()

0

### Check for null values

In [361]:
# Checking for null values
df_match_ed.isnull().sum()

season       0
stage        0
date         0
team_goal    0
side         0
country      0
league       0
team         0
dtype: int64

### Final check

In [362]:
df_match_ed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51955 entries, 0 to 51957
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   season     51955 non-null  object
 1   stage      51955 non-null  int64 
 2   date       51955 non-null  object
 3   team_goal  51955 non-null  int64 
 4   side       51955 non-null  object
 5   country    51955 non-null  object
 6   league     51955 non-null  object
 7   team       51955 non-null  object
dtypes: int64(2), object(6)
memory usage: 3.6+ MB


## **B. `Player` Table**

In [363]:
df_player.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182,154


> I don't need the `player_fifa_api_id` column so I drop it

In [364]:
# Dropping column
df_player_ed = df_player.drop(columns=['player_fifa_api_id'])


In [365]:
# Confirming changes
df_player_ed.head()

,id,player_api_id,player_name,birthday,height,weight
0,1,505942,Aaron Appindangoye,1992-02-29 00:00:00,182,187
1,2,155782,Aaron Cresswell,1989-12-15 00:00:00,170,146
2,3,162549,Aaron Doran,1991-05-13 00:00:00,170,163
3,4,30572,Aaron Galindo,1982-05-08 00:00:00,182,198
4,5,23780,Aaron Hughes,1979-11-08 00:00:00,182,154


> I would like to add player attributes to this table from the `player attributes` table so lets have a look at the table

In [366]:
df_player_attr.head(2)

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0


> There are many columns in this table that I will not be using for my analysis so I will be droopping them. I will only be choosing a few player attribures and dropping the rest. The attributes I will be keeping are: `overall_rating, preferred_foot, finishing, acceleration, sprint_speed and stamina`.

In [367]:
# Creating a function to store all columns that will be removed
def player_columns_to_remove():
    player_attr_columns = list(df_player_attr.columns)
    player_attr_columns_to_keep = ['player_api_id', 'date', 'overall_rating', 'preferred_foot', 'finishing', 'acceleration', 'sprint_speed', 'stamina']
    [player_attr_columns.remove(x) for x in player_attr_columns_to_keep]
    return player_attr_columns

In [368]:
# Dropping columns
df_player_attr.drop(columns=player_columns_to_remove(), inplace=True)

# Confirming changes
df_player_attr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183978 entries, 0 to 183977
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   player_api_id   183978 non-null  int64  
 1   date            183978 non-null  object 
 2   overall_rating  183142 non-null  float64
 3   preferred_foot  183142 non-null  object 
 4   finishing       183142 non-null  float64
 5   acceleration    183142 non-null  float64
 6   sprint_speed    183142 non-null  float64
 7   stamina         183142 non-null  float64
dtypes: float64(5), int64(1), object(2)
memory usage: 11.2+ MB


> Now we merge the `player` table with the `player_attr` table using the **player_api_key** foreign key

In [369]:
# Merging the tables
df_player_ed = pd.merge(df_player_ed, df_player_attr, on='player_api_id', how='inner')

# Confirming changes
df_player_ed.head()

,id,player_api_id,player_name,birthday,height,weight,date,overall_rating,preferred_foot,finishing,acceleration,sprint_speed,stamina
0,1,505942,Aaron Appindangoye,1992-02-29 00:00:00,182,187,2016-02-18 00:00:00,67.0,right,44.0,60.0,64.0,54.0
1,1,505942,Aaron Appindangoye,1992-02-29 00:00:00,182,187,2015-11-19 00:00:00,67.0,right,44.0,60.0,64.0,54.0
2,1,505942,Aaron Appindangoye,1992-02-29 00:00:00,182,187,2015-09-21 00:00:00,62.0,right,44.0,60.0,64.0,54.0
3,1,505942,Aaron Appindangoye,1992-02-29 00:00:00,182,187,2015-03-20 00:00:00,61.0,right,43.0,60.0,64.0,54.0
4,1,505942,Aaron Appindangoye,1992-02-29 00:00:00,182,187,2007-02-22 00:00:00,61.0,right,43.0,60.0,64.0,54.0


In [370]:
# Dropping the id column
df_player_ed.drop(columns=['id', 'player_api_id'], inplace=True)

# Confirming changes
df_player_ed.head()

,player_name,birthday,height,weight,date,overall_rating,preferred_foot,finishing,acceleration,sprint_speed,stamina
0,Aaron Appindangoye,1992-02-29 00:00:00,182,187,2016-02-18 00:00:00,67.0,right,44.0,60.0,64.0,54.0
1,Aaron Appindangoye,1992-02-29 00:00:00,182,187,2015-11-19 00:00:00,67.0,right,44.0,60.0,64.0,54.0
2,Aaron Appindangoye,1992-02-29 00:00:00,182,187,2015-09-21 00:00:00,62.0,right,44.0,60.0,64.0,54.0
3,Aaron Appindangoye,1992-02-29 00:00:00,182,187,2015-03-20 00:00:00,61.0,right,43.0,60.0,64.0,54.0
4,Aaron Appindangoye,1992-02-29 00:00:00,182,187,2007-02-22 00:00:00,61.0,right,43.0,60.0,64.0,54.0


### Checking for duplicate rows

In [373]:
df_player_ed.duplicated().sum()

1

In [ ]:
# Dropping the duplicate
df_player_ed = df_player_ed.drop_duplicates()

# Confirming changes
df_player_ed.duplicated().sum()

### Dealing with null values

In [374]:
df_player_ed.isnull().sum()

player_name         0
birthday            0
height              0
weight              0
date                0
overall_rating    836
preferred_foot    836
finishing         836
acceleration      836
sprint_speed      836
stamina           836
dtype: int64

### Final check

In [372]:
df_player_ed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183978 entries, 0 to 183977
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   player_name     183978 non-null  object 
 1   birthday        183978 non-null  object 
 2   height          183978 non-null  int64  
 3   weight          183978 non-null  int64  
 4   date            183978 non-null  object 
 5   overall_rating  183142 non-null  float64
 6   preferred_foot  183142 non-null  object 
 7   finishing       183142 non-null  float64
 8   acceleration    183142 non-null  float64
 9   sprint_speed    183142 non-null  float64
 10  stamina         183142 non-null  float64
dtypes: float64(5), int64(2), object(4)
memory usage: 16.8+ MB


# **Investigating The Data**

---


In [ ]:
# Checking Ronaldo
df_player_ed[df_player_ed.player_name == 'Cristiano Ronaldo']

Again, I will drop the `team_fifa_api_id` column

In [ ]:
# Dropping column
df_team_attr.drop(columns=['team_fifa_api_id'], inplace=True)

In [ ]:
# Confirming changes
df_team_attr.head()

### Adding columns
> Now I will be adding a column for `year` so that I can aggregte each teams attribute by year

In [ ]:
# Creating a column and extracting the year from the date column 
df_team_attr['year'] = df_team_attr['date'].apply(lambda x: x[0:4]).astype(int)

# Confirming changes
df_team_attr.head()

> I plan to add some columns to the `team` table from this table so I will be aggregating the table by the average attributes of each team for each year

In [ ]:
table = df_team_attr.groupby(['team_api_id', 'year']).mean()
table.head(50)

> I notice that there is incomplete data for some teams. Some teams have missing data for some years. So I descide to create tables for each year and seperate the team data into the year they fall on

In [ ]:
year_10 = df_team_attr[df_team_attr.year == 2010]
year_11 = df_team_attr[df_team_attr.year == 2011]
year_12 = df_team_attr[df_team_attr.year == 2012]
year_13 = df_team_attr[df_team_attr.year == 2013]
year_14 = df_team_attr[df_team_attr.year == 2014]
year_15 = df_team_attr[df_team_attr.year == 2015]
year_10.head()

In [ ]:
tab = year_10.groupby(['team_api_id']).mean()
plt.plot(tab['buildUpPlaySpeed'])


# **Conclusions**

---

> The dataset is not very well explained in the documentation and there are many columns in come of the tables which I do not know what they represent so I drop them and try to reduce the tables to something I can work with.

# Misc queries

In [ ]:
# df_team_attr.sort_values(by='team_api_id')

# df_team.query('team_long_name == "Manchester City"')
# df_team[df_team['team_long_name'] == 'Manchester City']
# df_team[df_team.team_long_name == 'Manchester City']

# df_team.query('team_fifa_api_id.isnull()')

# cursor.execute(""
#                "SELECT * FROM Team WHERE team_fifa_api_id is Null"""
#                )
# pprint.pprint(cursor.fetchall())

# df_match_ed[df_match_ed.home_team_api_id == 8456]


In [ ]:
# SQLite query to filter the match to return all Manchester City matches played in 2015 

# cursor.execute(""
#                "SELECT * FROM Match WHERE home_team_api_id=8456 AND SUBSTR(date, 1, 4) = '2015' """
#                )
# pprint.pprint(cursor.fetchone())